# Donna Life OS — `core.py` Deep Dive Tutorial

This notebook walks through every component of `src/core.py`, the heart of the Donna agent.
You'll see how configuration, system prompt assembly, permission handling, subagents,
skills, and the `DonnaAgent` class all fit together.

**What you'll learn:**
1. Configuration layer (`config.py`)
2. Helper functions — loading user data, timezone handling, date context
3. System prompt assembly — how Donna builds its full prompt
4. New-user detection & onboarding
5. Permission handling — the async callback pattern
6. The `DonnaAgent` class — async context manager, send/receive
7. Skills & Subagents — writing, onboarding, file-search, context-updater
8. The CLI interface — how `cli.py` wires it all up
9. Architecture recap

---
## 0. Setup — Imports & Path Configuration

We need to add `src/` to the Python path so we can import the modules
the same way the application does.

In [1]:
import sys, os
from pathlib import Path

# Add project root so we can do `from src.X import ...`
PROJECT_ROOT = Path.cwd()  # donna-life-os/
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print(f"Project root: {PROJECT_ROOT}")
print(f"src/ directory: {PROJECT_ROOT / 'src'}")

Project root: /home/user/donna-life-os
src/ directory: /home/user/donna-life-os/src


---
## 1. Configuration (`src/config.py`)

All tunables live in one file. Let's inspect them.

In [2]:
from src.config import (
    MODEL,
    MAX_THINKING_TOKENS,
    ALLOWED_TOOLS,
    AUTO_ALLOWED_TOOLS,
    DONNA_DATA_DIR,
    CURRENT_CONTEXT_FILE,
    USER_PREFERENCES_FILE,
    MAX_BUDGET_USD,
    MAX_TURNS,
)

print("=== Model Configuration ===")
print(f"  MODEL              : {MODEL!r}")
print(f"  MAX_THINKING_TOKENS: {MAX_THINKING_TOKENS}")
print(f"  MAX_BUDGET_USD     : {MAX_BUDGET_USD}")
print(f"  MAX_TURNS          : {MAX_TURNS}")

print("\n=== Tool Configuration ===")
print(f"  ALLOWED_TOOLS      : {ALLOWED_TOOLS}")
print(f"  AUTO_ALLOWED_TOOLS : {AUTO_ALLOWED_TOOLS}")

print("\n=== Data Paths ===")
print(f"  DONNA_DATA_DIR         : {DONNA_DATA_DIR}")
print(f"  CURRENT_CONTEXT_FILE   : {CURRENT_CONTEXT_FILE}")
print(f"  USER_PREFERENCES_FILE  : {USER_PREFERENCES_FILE}")
print(f"  donna-data exists?     : {DONNA_DATA_DIR.exists()}")

=== Model Configuration ===
  MODEL              : 'opus'
  MAX_THINKING_TOKENS: 10000
  MAX_BUDGET_USD     : None
  MAX_TURNS          : None

=== Tool Configuration ===
  ALLOWED_TOOLS      : ['Read', 'Write', 'Edit', 'Skill', 'Grep', 'Glob', 'Task']
  AUTO_ALLOWED_TOOLS : ['Read', 'Write', 'Edit', 'Grep', 'Glob', 'Skill', 'Task']

=== Data Paths ===
  DONNA_DATA_DIR         : /root/donna-data
  CURRENT_CONTEXT_FILE   : /root/donna-data/current_context.md
  USER_PREFERENCES_FILE  : /root/donna-data/user_info_and_preferences.md
  donna-data exists?     : False


### Key takeaways

| Setting | Purpose |
|---|---|
| `ALLOWED_TOOLS` | Tools the agent CAN use (Read, Write, Edit, Skill, Grep, Glob, Task). Note: `Bash` is intentionally omitted — it requires explicit user permission via the callback. |
| `AUTO_ALLOWED_TOOLS` | Tools that never prompt the user. All the safe, data-focused tools are here. |
| `DONNA_DATA_DIR` | `~/donna-data/` — the user's sovereign data folder. Plain markdown files they own. |
| `MAX_THINKING_TOKENS` | Extended thinking budget (10k tokens) for complex reasoning. |

---
## 2. Helper Functions — Loading User Data

`core.py` has several functions that load data from `~/donna-data/` to build context for the agent.

### 2a. `load_user_info_and_preferences()`

Parses the YAML frontmatter from `user_info_and_preferences.md`.
This file stores the user's name, timezone, communication style, and personal/professional context.

In [3]:
from src.core import load_user_info_and_preferences, get_user_name, get_user_timezone

prefs = load_user_info_and_preferences()
print("User preferences (YAML frontmatter):")
if prefs:
    for k, v in prefs.items():
        print(f"  {k}: {v}")
else:
    print("  (empty — no preferences file found or no frontmatter)")

print(f"\nget_user_name()     → {get_user_name()!r}")
print(f"get_user_timezone() → {get_user_timezone()!r}")

User preferences (YAML frontmatter):
  (empty — no preferences file found or no frontmatter)

get_user_name()     → 'there'
get_user_timezone() → None


### 2b. `load_current_context()`

Reads `current_context.md` — Donna's working memory of what the user is currently focused on.
This gets injected into the system prompt so Donna can reference active topics.

In [4]:
from src.core import load_current_context

ctx = load_current_context()
if ctx.strip():
    print("Current context (first 500 chars):")
    print(ctx[:500])
    if len(ctx) > 500:
        print(f"\n... ({len(ctx)} total chars)")
else:
    print("No current context found (empty or file missing).")
    print("This is normal for a new user or fresh install.")

No current context found (empty or file missing).
This is normal for a new user or fresh install.


### 2c. `is_new_user()` and `setup_donna_data_directory()`

New-user detection drives the onboarding flow:
- If `user_info_and_preferences.md` doesn't exist or has no `name` → new user
- `setup_donna_data_directory()` copies the template folder structure to `~/donna-data/`

In [5]:
from src.core import is_new_user, setup_donna_data_directory

print(f"is_new_user() → {is_new_user()}")
print(f"\nPreferences file exists? {USER_PREFERENCES_FILE.exists()}")

# Show what the template directory contains
template_path = PROJECT_ROOT / "src" / ".claude" / "skills" / "onboarding" / "template-donna-data"
if template_path.exists():
    print(f"\nTemplate directory structure ({template_path}):")
    for p in sorted(template_path.rglob("*")):
        relative = p.relative_to(template_path)
        prefix = "  📁 " if p.is_dir() else "  📄 "
        print(f"{prefix}{relative}")
else:
    print("Template directory not found.")

is_new_user() → True

Preferences file exists? False

Template directory structure (/home/user/donna-life-os/src/.claude/skills/onboarding/template-donna-data):
  📁 check-ins
  📄 check-ins/_template.md
  📄 current_context.md
  📁 daily-logs
  📄 daily-logs/_template.md
  📁 goals
  📄 goals/_template.md
  📁 notes
  📄 notes/_template.md
  📁 people
  📄 people/_template.md
  📁 projects
  📄 projects/_template.md
  📁 tasks
  📄 tasks/_template.md
  📄 user_info_and_preferences.md


---
## 3. Timezone & Date Context

Donna injects rich date/time context into every system prompt so it can reason about
deadlines, scheduling, and relative dates ("next Tuesday", "end of month").

### 3a. `get_effective_timezone()` — Timezone Fallback Chain

Three-level fallback:
1. Client-provided timezone (from browser, e.g. web interface)
2. User's stored timezone preference
3. System local timezone

In [6]:
from src.core import get_effective_timezone

# No client timezone — falls back through the chain
tz_default = get_effective_timezone()
print(f"Default (no client TZ) → {tz_default}")

# Simulate a browser sending its timezone
tz_client = get_effective_timezone("America/New_York")
print(f"With client TZ         → {tz_client}")

# Invalid timezone gracefully falls through
tz_bad = get_effective_timezone("Invalid/Timezone")
print(f"Bad client TZ fallback → {tz_bad}")

Default (no client TZ) → Etc/UTC
With client TZ         → America/New_York
Bad client TZ fallback → Etc/UTC


### 3b. `generate_date_context()` — What Gets Injected

This generates a formatted block with today's date, the next 7 days, and reference
points (1 week, 2 weeks, end of month). This is what Claude sees in its system prompt.

In [7]:
from src.core import generate_date_context

date_ctx = generate_date_context()
print(date_ctx)

═══ DATE & TIME CONTEXT ═══
Today: Thursday, January 29, 2026
Current time: 11:33 PM UTC (Etc/UTC) [2026-01-29T23:33:01.809425+00:00]

─── This Week ───
Tomorrow (Fri):    January 30, 2026 [2026-01-30]
Saturday:          January 31, 2026 [2026-01-31]
Sunday:            February 1, 2026 [2026-02-01]
Next Monday:       February 2, 2026 [2026-02-02]
Next Tuesday:      February 3, 2026 [2026-02-03]
Next Wednesday:    February 4, 2026 [2026-02-04]
Next Thursday:     February 5, 2026 [2026-02-05]

─── Reference Points ───
1 week from now:   Thursday, February 5, 2026 [2026-02-05]
2 weeks from now:  Thursday, February 12, 2026 [2026-02-12]
End of month:      Saturday, January 31, 2026 [2026-01-31]


---
## 4. System Prompt Assembly

`build_full_system_prompt()` is the function that stitches everything together into the
final system prompt Claude receives. It combines:

1. **Base personality** from `src/prompt.md` — Donna's identity, rules, and behavior
2. **Date/time context** — generated dynamically
3. **User preferences** — name, communication style, personal/professional context
4. **Current context** — active topics, working memory

In [8]:
from src.core import build_full_system_prompt, load_system_prompt

# First, let's see the base prompt size
base = load_system_prompt()
print(f"Base prompt (prompt.md): {len(base)} chars, {len(base.splitlines())} lines")
print(f"First 200 chars:\n{base[:200]}...")

Base prompt (prompt.md): 8271 chars, 189 lines
First 200 chars:
# Donna - Your Life Operating System

You are Donna, an AI-native life operating system. You help humans stay in flow by handling the organizational overhead they'd rather not think about.

## Core Pu...


In [9]:
# Now the full assembled prompt
full_prompt = build_full_system_prompt()

print(f"Full system prompt: {len(full_prompt)} chars, {len(full_prompt.splitlines())} lines")
print(f"\nSections in the full prompt:")
print("─" * 50)

# Show the section headers to illustrate structure
for i, line in enumerate(full_prompt.splitlines()):
    if line.startswith("## ") or line.startswith("# ") or line.startswith("═══"):
        print(f"  Line {i+1:>3}: {line}")

Full system prompt: 9068 chars, 213 lines

Sections in the full prompt:
──────────────────────────────────────────────────
  Line   1: # Donna - Your Life Operating System
  Line   5: ## Core Purpose
  Line   9: ## Personality
  Line  15: ## Critical Behavior: Capture First (MOST IMPORTANT)
  Line  42: # Tax thing to deal with
  Line  46: ## Open Questions
  Line  51: ## Reasoning
  Line  59: ## Data Model
  Line  90: ## Preserving History
  Line  96: ## 2026-01-16 - Deadline extended
  Line 104: ## Tools
  Line 112: ## Finding Existing Context
  Line 139: ## Current Context Management
  Line 179: ## What NOT to Do
  Line 192: ═══ DATE & TIME CONTEXT ═══
  Line 211: ## Current Active Context


In [10]:
# Show the injected context sections (the dynamic parts)
# These appear AFTER the base prompt.md content
base_end = len(base)
injected = full_prompt[base_end:]
print("=== Dynamically Injected Context ===")
print(injected[:2000] if len(injected) > 2000 else injected)

=== Dynamically Injected Context ===


═══ DATE & TIME CONTEXT ═══
Today: Thursday, January 29, 2026
Current time: 11:33 PM UTC (Etc/UTC) [2026-01-29T23:33:01.826471+00:00]

─── This Week ───
Tomorrow (Fri):    January 30, 2026 [2026-01-30]
Saturday:          January 31, 2026 [2026-01-31]
Sunday:            February 1, 2026 [2026-02-01]
Next Monday:       February 2, 2026 [2026-02-02]
Next Tuesday:      February 3, 2026 [2026-02-03]
Next Wednesday:    February 4, 2026 [2026-02-04]
Next Thursday:     February 5, 2026 [2026-02-05]

─── Reference Points ───
1 week from now:   Thursday, February 5, 2026 [2026-02-05]
2 weeks from now:  Thursday, February 12, 2026 [2026-02-12]
End of month:      Saturday, January 31, 2026 [2026-01-31]


## Current Active Context

No active context items. This may be a new user or a fresh start.



---
## 5. Greeting Generation

When a session starts, `DonnaAgent` automatically sends a greeting.
The greeting differs based on whether this is a new or returning user.

In [11]:
from src.core import generate_greeting_prompt

greeting = generate_greeting_prompt()
is_onboarding = "ONBOARDING" in greeting

print(f"Greeting type: {'ONBOARDING (new user)' if is_onboarding else 'RETURNING USER'}")
print(f"\nGreeting prompt sent to Claude:")
print("─" * 50)
print(greeting)

Greeting type: ONBOARDING (new user)

Greeting prompt sent to Claude:
──────────────────────────────────────────────────
[SYSTEM - ONBOARDING]
This is a new user. The ~/donna-data folder structure has been initialized.
Use the new-user-onboarding skill to guide them through a natural introduction. I am now going to invoke the new-user-onboarding skill.


**How it works:**
- New user → triggers the `new-user-onboarding` skill (guided intro)
- Returning user → generates a contextual greeting using their name and current context

The greeting is the **first message** in every session. The interface just calls
`receive_response()` to stream the agent's greeting back to the user.

---
## 6. Permission Handling — The Async Callback Pattern

The `PermissionRequest` class is the bridge between the agent wanting to use a tool
and the interface asking the user for approval.

This is a clean **inversion of control**: the core agent doesn't know *how* to ask for
permission — it just emits a request and waits for a response.

In [12]:
import asyncio
from src.core import PermissionRequest

# Simulate a permission request (like Bash command)
# In real use, the SDK calls the permission handler internally

async def demo_permission():
    # Create a permission request as the agent would
    request = PermissionRequest(
        tool_name="Bash",
        tool_input={"command": "ls ~/donna-data/tasks/"}
    )
    print(f"Tool requesting permission: {request.tool_name}")
    print(f"Tool input: {request.tool_input}")

    # Simulate the interface allowing it
    await request.allow()
    result = await request.wait_for_response()
    print(f"\nPermission result: {result}")
    print(f"Result type: {type(result).__name__}")

await demo_permission()

Tool requesting permission: Bash
Tool input: {'command': 'ls ~/donna-data/tasks/'}

Permission result: PermissionResultAllow(behavior='allow', updated_input=None, updated_permissions=None)
Result type: PermissionResultAllow


In [13]:
# Demonstrate a denial
async def demo_denial():
    request = PermissionRequest(
        tool_name="Bash",
        tool_input={"command": "rm -rf /"}
    )
    await request.deny("That looks dangerous!")
    result = await request.wait_for_response()
    print(f"Permission result: {result}")
    print(f"Result type: {type(result).__name__}")

await demo_denial()

Permission result: PermissionResultDeny(behavior='deny', message='That looks dangerous!', interrupt=False)
Result type: PermissionResultDeny


### The Permission Flow in DonnaAgent

```
Agent wants to use a tool
        │
        ▼
┌─────────────────────────────┐
│ _permission_handler()       │
│  (internal to DonnaAgent)   │
└─────────────────────────────┘
        │
    Is tool in AUTO_ALLOWED_TOOLS?
        │
   Yes ─┤─ No
   │    │    │
   │    │    ▼
   │    │  Is there a callback?
   │    │    │
   │    │  No ──→ Auto-allow
   │    │    │
   │    │  Yes
   │    │    │
   │    │    ▼
   │    │  Call on_permission_request()
   │    │  (provided by the interface)
   │    │    │
   ▼    │    ▼
 Allow   │  Allow / Deny
```

The CLI provides `cli_permission_handler` which prompts the user on the terminal.
The web interface could use a WebSocket message instead. The core agent doesn't care.

---
## 7. The `DonnaAgent` Class

This is the main class. Let's examine its structure without running it
(since it needs an API key and a live Claude session).

In [14]:
import inspect
from src.core import DonnaAgent

# Show the class signature and docstring
print("class DonnaAgent")
print("=" * 50)
print(inspect.getdoc(DonnaAgent))

print("\n\nPublic Methods:")
print("─" * 50)
for name, method in inspect.getmembers(DonnaAgent, predicate=inspect.isfunction):
    if not name.startswith("_") or name in ("__init__", "__aenter__", "__aexit__"):
        sig = inspect.signature(method)
        doc = inspect.getdoc(method) or ""
        first_line = doc.split("\n")[0] if doc else "(no docstring)"
        print(f"\n  {name}{sig}")
        print(f"    → {first_line}")

class DonnaAgent
Core Donna agent that can be used with any interface.

Uses an async callback pattern for permission handling, allowing
different interfaces (CLI, web, etc.) to handle permissions their own way.

Example usage:
    async def my_permission_handler(request: PermissionRequest) -> bool:
        # Your logic to ask user for permission
        return True  # or False
    
    async with DonnaAgent(on_permission_request=my_permission_handler) as donna:
        # Agent automatically sends a greeting - just receive it
        async for message in donna.receive_response():
            print(message)
        
        # Then continue with normal conversation
        await donna.send_message("Hello!")
        async for message in donna.receive_response():
            print(message)


Public Methods:
──────────────────────────────────────────────────

  __aenter__(self) -> 'DonnaAgent'
    → Enter the async context manager, initializing the Claude client.

  __aexit__(self, exc_type

In [15]:
# Show the __init__ parameters in detail
print("DonnaAgent.__init__ parameters:")
print("─" * 50)
sig = inspect.signature(DonnaAgent.__init__)
for name, param in sig.parameters.items():
    if name == "self":
        continue
    default = param.default if param.default != inspect.Parameter.empty else "(required)"
    print(f"  {name}: {param.annotation.__name__ if hasattr(param.annotation, '__name__') else param.annotation}")
    print(f"    default = {default}")

DonnaAgent.__init__ parameters:
──────────────────────────────────────────────────
  on_permission_request: Optional
    default = None
  model: str | None
    default = None
  allowed_tools: list[str] | None
    default = None
  max_thinking_tokens: int | None
    default = None
  auto_greet: bool
    default = True
  client_timezone: str | None
    default = None


### How `DonnaAgent` works (lifecycle)

```python
# 1. Create with optional permission callback
async with DonnaAgent(on_permission_request=my_handler) as donna:
    
    # 2. __aenter__ runs:
    #    - Builds full system prompt (personality + user data + date context)
    #    - Creates ClaudeSDKClient with options (model, tools, cwd=src/)
    #    - Sends auto-greeting (onboarding or returning-user greeting)
    
    # 3. Receive the greeting
    async for message in donna.receive_response():
        print(message)  # Stream the greeting
    
    # 4. Conversation loop
    await donna.send_message("I need to plan a birthday party")
    async for message in donna.receive_response():
        print(message)  # Stream the response

# 5. __aexit__ cleans up the SDK client
```

Key design decisions:
- **Async context manager**: Ensures cleanup even on errors
- **`cwd=src/`**: Makes the SDK load skills from `src/.claude/`, not the project root
- **Streaming input**: Uses `create_user_message()` async generator for SDK compatibility
- **Auto-greeting**: The agent speaks first, creating a proactive experience

In [16]:
# Let's look at the ClaudeAgentOptions that get built in __aenter__
# We'll reconstruct what happens without actually connecting

from src.core import build_full_system_prompt
from src.config import ALLOWED_TOOLS, MAX_THINKING_TOKENS, MAX_BUDGET_USD, MAX_TURNS

src_dir = str(PROJECT_ROOT / "src")

print("ClaudeAgentOptions built in __aenter__:")
print("─" * 50)
print(f"  model            : {MODEL!r}")
print(f"  allowed_tools    : {ALLOWED_TOOLS}")
print(f"  cwd              : {src_dir!r}")
print(f"  setting_sources  : ['project']")
print(f"  max_thinking     : {MAX_THINKING_TOKENS}")
print(f"  max_budget_usd   : {MAX_BUDGET_USD}")
print(f"  max_turns        : {MAX_TURNS}")
print(f"  system_prompt    : ({len(build_full_system_prompt())} chars)")
print(f"  can_use_tool     : <permission_handler function>")

ClaudeAgentOptions built in __aenter__:
──────────────────────────────────────────────────
  model            : 'opus'
  allowed_tools    : ['Read', 'Write', 'Edit', 'Skill', 'Grep', 'Glob', 'Task']
  cwd              : '/home/user/donna-life-os/src'
  setting_sources  : ['project']
  max_thinking     : 10000
  max_budget_usd   : None
  max_turns        : None
  system_prompt    : (10792 chars)
  can_use_tool     : <permission_handler function>


### Streaming Input: `create_user_message()`

The SDK expects messages as async generators. This simple function wraps
plain text into the required format.

In [17]:
from src.core import create_user_message

async def show_message_format():
    async for msg in create_user_message("Hello Donna!"):
        print("Message yielded by create_user_message():")
        import json
        print(json.dumps(msg, indent=2))

await show_message_format()

Message yielded by create_user_message():
{
  "type": "user",
  "message": {
    "role": "user",
    "content": "Hello Donna!"
  }
}


---
## 8. Skills — Reusable Capabilities

Skills live in `src/.claude/skills/` and are loaded by the SDK when `cwd=src/`.
They are markdown files that define behaviors Donna can invoke.

### Skill Architecture

```
src/.claude/skills/
├── onboarding/
│   ├── SKILL.md                  ← Skill definition
│   └── template-donna-data/      ← Template folder for new users
│       ├── tasks/_template.md
│       ├── projects/_template.md
│       ├── notes/_template.md
│       └── ...
└── writing/
    ├── SKILL.md                  ← Skill definition
    └── content-types.md          ← Content type reference
```

In [18]:
# Read and display the skill definitions
skills_dir = PROJECT_ROOT / "src" / ".claude" / "skills"

for skill_dir in sorted(skills_dir.iterdir()):
    if not skill_dir.is_dir():
        continue
    skill_file = skill_dir / "SKILL.md"
    if skill_file.exists():
        content = skill_file.read_text()
        # Extract frontmatter
        if content.startswith("---"):
            parts = content.split("---", 2)
            frontmatter = parts[1].strip()
            body_preview = parts[2].strip()[:200]
        else:
            frontmatter = "(no frontmatter)"
            body_preview = content[:200]
        
        print(f"━━━ Skill: {skill_dir.name} ━━━")
        print(f"Frontmatter:\n{frontmatter}")
        print(f"\nBody preview:\n{body_preview}...")
        print()

━━━ Skill: onboarding ━━━
Frontmatter:
name: new-user-onboarding
description: Guide new users through a natural introduction to Donna

Body preview:
# Onboarding Skill

This skill guides first-time users through a friendly introduction to Donna, getting to know them personally and professionally before explaining what Donna can do.

## Note: Data ...

━━━ Skill: writing ━━━
Frontmatter:
name: writing
description: Guidelines for creating new markdown files in donna-data

Body preview:
# Writing Skill

This skill provides guidelines for creating new markdown files in `~/donna-data/`.

## Content Type Definitions

**Before creating any content, consult [content-types.md](./content-ty...



### Onboarding Skill — New User Flow

When `is_new_user()` returns `True`, the greeting prompt triggers the onboarding skill.
The flow:

1. Ask for name → **immediately write** to `user_info_and_preferences.md`
2. Ask about personal life → **immediately update** the file
3. Ask about professional side → **immediately update** the file
4. Ask about communication style → **immediately save** preference
5. Explain what Donna can do

Critical design choice: **save immediately, not at the end**. Conversations can drop at any time.

### Writing Skill — File Creation Guidelines

Before creating ANY file in `~/donna-data/`, Donna must invoke the writing skill.
This ensures:
- Correct content type is chosen (Task vs Project vs Note vs Goal)
- Template files (`_template.md`) are consulted for structure
- Consistent frontmatter and formatting across all files

In [19]:
# Show the content type decision flowchart
content_types_file = skills_dir / "writing" / "content-types.md"
content = content_types_file.read_text()

# Extract just the flowchart and summary table
lines = content.split("\n")
in_flowchart = False
flowchart = []
for line in lines:
    if "Decision Flowchart" in line:
        in_flowchart = True
    elif in_flowchart and line.startswith("---"):
        break
    if in_flowchart:
        flowchart.append(line)

print("\n".join(flowchart))

# Show summary table
print("\n" + "═" * 60)
in_summary = False
for line in lines:
    if "Content Type at a Glance" in line:
        in_summary = True
    if in_summary:
        print(line)

## Decision Flowchart

```
User mentions something
         │
         ▼
┌─────────────────────────┐
│ Is there a clear action?│
└─────────────────────────┘
         │
    No   │   Yes
    ▼    │    ▼
  NOTE   │  ┌─────────────────────────┐
         │  │ Is it multi-step with   │
         │  │ deadline/event/phases?  │
         │  └─────────────────────────┘
         │         │
         │    No   │   Yes
         │    ▼    │    ▼
         │  TASK   │  PROJECT + suggest tasks
         │         │
         ▼─────────┴─────────▼
                   │
    Also update CURRENT CONTEXT if relevant
```


════════════════════════════════════════════════════════════
## Summary: Content Type at a Glance

| Type | Core Question | Key Signal |
|------|---------------|------------|
| Task | "What single action needs doing?" | Verb + object, completable in one sitting |
| Project | "What multi-step effort is this?" | Event, launch, transition, "planning X" |
| Note | "What's worth remembering?" | No 

---
## 9. Subagents — Delegated Specialists

Subagents live in `src/.claude/agents/` and handle specific tasks that benefit from
a focused, specialized agent. They're spawned via the `Task` tool.

```
src/.claude/agents/
├── file-search.md      ← Fast file discovery (Haiku model)
└── context-updater.md   ← Maintains working memory (Sonnet model)
```

In [20]:
# Read and display agent definitions
agents_dir = PROJECT_ROOT / "src" / ".claude" / "agents"

for agent_file in sorted(agents_dir.glob("*.md")):
    content = agent_file.read_text()
    # Extract frontmatter
    if content.startswith("---"):
        parts = content.split("---", 2)
        frontmatter = parts[1].strip()
    else:
        frontmatter = "(no frontmatter)"
    
    print(f"━━━ Agent: {agent_file.stem} ━━━")
    print(f"Frontmatter:\n{frontmatter}")
    print()

━━━ Agent: context-updater ━━━
Frontmatter:
name: context-updater
description: Updates ~/donna-data/current_context.md with recently discussed topics. Run this after any substantive conversation to keep working memory fresh.
tools: Read, Write, Edit, Glob, Grep
model: sonnet

━━━ Agent: file-search ━━━
Frontmatter:
name: file-search
description: Fast, authoritative, date-aware file search for ~/donna-data/ with simple (default) and exhaustive modes. Returns file paths and full contents. The main agent relies on this for all file discovery.
tools: Read, Glob, Grep, Bash
model: haiku



### file-search Agent

The main agent's go-to for finding existing files. Has two modes:

| Mode | Trigger | Behavior |
|------|---------|----------|
| **Simple** (default) | Any search request | Single Glob lookup → auto-escalate if nothing found |
| **Exhaustive** | "exhaustive", "find everything", auto-escalation | Multi-strategy: Glob + Grep + content search + date search |

**Why Haiku model?** File search is a fast, mechanical task — no deep reasoning needed.
Using Haiku keeps it cheap and fast.

**Wikilink resolution:**
```
[[Baby Shower]]     → baby-shower.md
[[Tax Filing 2026]] → tax-filing-2026.md
[[Sarah]]           → sarah.md
```

**How the main agent spawns it:**
```python
# The agent uses the Task tool internally:
Task(
    subagent_type="file-search",
    prompt='Find [[Baby Shower]]',
    run_in_background=False  # Need results before responding
)
```

### context-updater Agent

Maintains `current_context.md` — Donna's working memory. Runs **asynchronously**
after substantive conversations.

**Rules:**
- Keep 5-10 active items max
- Set `last_mentioned` dates (ISO 8601)
- Remove items older than 30 days
- Remove resolved items
- Use wikilinks for all entity references

**Why Sonnet model?** Context updating needs better reasoning to decide what to
keep, update, or remove.

**How the main agent spawns it:**
```python
# Runs in background — don't wait for it
Task(
    subagent_type="context-updater",
    run_in_background=True,
    prompt="""CONVERSATION SUMMARY:
User discussed planning a birthday party for their daughter.

WIKILINKS TO USE:
- [[Birthday Party]]
- [[Emma]]

FILES TOUCHED:
- ~/donna-data/projects/birthday-party.md (created)

USER STATE:
Excited but slightly overwhelmed with planning."""
)
```

---
## 10. The CLI Interface (`src/cli.py`)

The CLI is one concrete interface built on top of `DonnaAgent`.
It demonstrates how any interface plugs in.

In [21]:
# Show the key components of cli.py
from src.cli import (
    print_thinking,
    print_tool_call,
    print_tool_result,
    print_session_summary,
    cli_permission_handler,
)

print("CLI Display Functions:")
print("─" * 50)
for fn in [print_thinking, print_tool_call, print_tool_result, print_session_summary]:
    doc = inspect.getdoc(fn) or ""
    print(f"  {fn.__name__}()")
    print(f"    → {doc.split(chr(10))[0]}")

print(f"\n  cli_permission_handler()")
print(f"    → {inspect.getdoc(cli_permission_handler).split(chr(10))[0]}")

CLI Display Functions:
──────────────────────────────────────────────────
  print_thinking()
    → Display agent's reasoning/thinking in a subtle purple panel.
  print_tool_call()
    → Display a tool call with its inputs in a yellow panel.
  print_tool_result()
    → Display tool execution result in green (success) or red (error) panel.
  print_session_summary()
    → Display session summary with cost and usage info.

  cli_permission_handler()
    → CLI-specific permission handler using Rich prompts.


In [22]:
# Demo the Rich UI components (these would normally show in the terminal)
from rich.console import Console
console = Console(width=80)

print("── Thinking Panel ──")
print_thinking("The user mentioned a birthday party. I should search for existing files first...")

print("\n── Tool Call Panel ──")
print_tool_call("Glob", {"pattern": "**/birthday*.md", "path": "~/donna-data"})

print("\n── Tool Result Panel (success) ──")
print_tool_result("Found: ~/donna-data/projects/birthday-party.md")

print("\n── Tool Result Panel (error) ──")
print_tool_result("File not found: ~/donna-data/tasks/nonexistent.md", is_error=True)

── Thinking Panel ──


╭────────────────────────────────────────────────── 🧠 Thinking ──────────────────────────────────────────────────╮
│ The user mentioned a birthday party. I should search for existing files first...                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


── Tool Call Panel ──


╭───────────────────────────────────────────────── 🔧 Tool: Glob ─────────────────────────────────────────────────╮
│ {                                                                                                               │
│   "pattern": "**/birthday*.md",                                                                                 │
│   "path": "~/donna-data"                                                                                        │
│ }                                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


── Tool Result Panel (success) ──


╭──────────────────────────────────────────────── 📤 Tool Result ─────────────────────────────────────────────────╮
│ Found: ~/donna-data/projects/birthday-party.md                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


── Tool Result Panel (error) ──


╭───────────────────────────────────────────────── ❌ Tool Error ─────────────────────────────────────────────────╮
│ File not found: ~/donna-data/tasks/nonexistent.md                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

### CLI Chat Loop

The `run_chat()` function is the main entry point. Here's the simplified flow:

```python
async def run_chat():
    # Show welcome banner
    console.print("Donna - Your Life Operating System")
    
    # Create agent with CLI-specific permission handler
    async with DonnaAgent(on_permission_request=cli_permission_handler) as donna:
        
        # Agent auto-greets → stream the response
        await display_response(donna)
        
        # Chat loop
        while True:
            user_input = await prompt_async("You: ")
            
            if user_input in ["exit", "quit", "q"]:
                # Update context file before exiting
                await _update_context_and_exit(donna)
                break
            
            await donna.send_message(user_input)
            await display_response(donna)
```

Notable: On exit, `_update_context_and_exit()` sends a system message asking Donna
to update `current_context.md` before closing. This keeps the working memory fresh
for the next session.

---
## 11. Architecture Recap — How It All Fits Together

```
┌──────────────────────────────────────────────────────────────────┐
│                        User Interfaces                          │
│  ┌──────────┐  ┌──────────┐  ┌──────────────────────────────┐  │
│  │  CLI      │  │  Web     │  │  Any future interface...     │  │
│  │ (cli.py)  │  │ (web/)   │  │  (mobile app, Slack bot...) │  │
│  └─────┬─────┘  └─────┬────┘  └──────────────┬───────────────┘  │
│        │              │                       │                  │
│        │   Each provides a permission_callback                  │
│        └──────────────┼───────────────────────┘                  │
│                       │                                          │
│                       ▼                                          │
│  ┌─────────────────────────────────────────┐                    │
│  │            DonnaAgent (core.py)          │                    │
│  │                                         │                    │
│  │  • Builds system prompt                 │                    │
│  │  • Manages ClaudeSDKClient              │                    │
│  │  • Handles permissions                  │                    │
│  │  • send_message() / receive_response()  │                    │
│  └────────────────────┬────────────────────┘                    │
│                       │                                          │
│                       ▼                                          │
│  ┌─────────────────────────────────────────┐                    │
│  │          Claude Agent SDK               │                    │
│  │                                         │                    │
│  │  Loads from cwd=src/:                   │                    │
│  │  • src/.claude/skills/   (Skills)       │                    │
│  │  • src/.claude/agents/   (Subagents)    │                    │
│  └────────────────────┬────────────────────┘                    │
│                       │                                          │
│         ┌─────────────┼─────────────┐                            │
│         ▼             ▼             ▼                            │
│  ┌───────────┐ ┌───────────┐ ┌───────────┐                     │
│  │  Skills   │ │ Subagents │ │   Tools   │                     │
│  │           │ │           │ │           │                     │
│  │ onboarding│ │file-search│ │ Read      │                     │
│  │ writing   │ │context-   │ │ Write     │                     │
│  │           │ │ updater   │ │ Edit      │                     │
│  └───────────┘ └───────────┘ │ Grep/Glob │                     │
│                              │ Bash      │                     │
│                              │ Skill     │                     │
│                              │ Task      │                     │
│                              └─────┬─────┘                     │
│                                    │                            │
│                                    ▼                            │
│                          ┌──────────────────┐                   │
│                          │   ~/donna-data/   │                   │
│                          │                  │                   │
│                          │ tasks/           │                   │
│                          │ projects/        │                   │
│                          │ notes/           │                   │
│                          │ people/          │                   │
│                          │ goals/           │                   │
│                          │ check-ins/       │                   │
│                          │ daily-logs/      │                   │
│                          │ current_context  │                   │
│                          │ user_preferences │                   │
│                          └──────────────────┘                   │
└──────────────────────────────────────────────────────────────────┘
```

### Key Design Principles in the Code

| Principle | Implementation |
|-----------|---------------|
| **Interface-agnostic core** | `DonnaAgent` knows nothing about CLI/web — just async send/receive |
| **Permission inversion** | Core emits `PermissionRequest`, interface decides how to ask the user |
| **Two `.claude/` scopes** | `src/.claude/` for runtime skills, root `.claude/` for dev tools |
| **Capture first** | Write files immediately, ask questions after — nothing gets lost |
| **Sovereign data** | All user data in `~/donna-data/` as plain markdown — user owns it |
| **Smart model selection** | Opus for main agent, Sonnet for context-updater, Haiku for file-search |
| **Auto-greet** | Agent speaks first — proactive, not reactive |
| **Context on exit** | Session end triggers context update so next session has fresh state |

### Data Flow Example: User says "I need to plan a birthday party"

```
1. User types message in CLI/web
     │
     ▼
2. donna.send_message("I need to plan a birthday party")
     │
     ▼
3. Claude receives message + full system prompt
   (personality + user prefs + current context + date info)
     │
     ▼
4. Agent spawns file-search subagent (Haiku)
   → "Find anything about birthday party"
   → Returns: no existing files found
     │
     ▼
5. Agent invokes writing skill
   → Determines this is a PROJECT (multi-step, event)
   → Reads projects/_template.md
   → Creates ~/donna-data/projects/birthday-party.md
     │
     ▼
6. Agent spawns context-updater subagent (Sonnet, background)
   → Updates current_context.md with [[Birthday Party]]
     │
     ▼
7. Agent responds to user:
   "Created [[Birthday Party]] project! Here are some tasks
    you might need — which should I add?
    - Create guest list
    - Send invitations
    - Book venue
    - Plan food & drinks"
```

---

That's the full `core.py` walkthrough. The design is intentionally simple:
one class, a handful of helper functions, and markdown-based skills/agents
that the SDK loads automatically. The complexity lives in the prompts and
the agent's reasoning, not in the code.